# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверить четыре гипотезы:
1. Семейное положение влияет на факт погашения кредита в срок.
2. Количество детей влияет на факт погашения кредита в срок.
3. Семейное положение влияет на факт погашения кредита в срок.
4. Уровень дохода влияет на факт погашения кредита в срок.

**Ход исследования**

Входные данные от банка - статистика о платёжеспособности клиентов получены из файла `/datasets/data.csv`. О качестве данных ничего не известно, поэтому перед проверкой гипотез понадобится обзор данных. 

Мы проверим данные на ошибки, а затем оценим их влияние на исследование. Затем, на этапе предобработки исправим ошибки в данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.


## Шаг 1. Откроем файл с данными и изучим общую информацию

Составим первое представление о полученных данных.

Основной инструмент аналитика - `pandas`. Импортируем эту библиотеку

In [1]:
#импорт библиотеки pandas
import pandas as pd

Прочитаем файл `data.csv` из папки `/datasets` и сохраним его в переменной `df`:

In [2]:
#чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

Выведим на экран первые десять строк таблицы:

In [3]:
# получение первых 10 строк таблицы df
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Одной командой получим общую информацию о таблице:

In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов с различным типом данных: `int64`, `float64`, `object`.
Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о клиентах банка. Часть колонок описывает социально-демографические характеристики клиентов. Остальные данные демонстрируют его финансовое состояние и способность вовремя оплачивать платежи по кредиту.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. С другой стороны, встречаются пропуски в данных. Столбец `days_employed` имеет нехарактерные отрицательные значения. Столбцы `days_employed` и `total_income` имеют нехарактерные для них типы данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Сначала посчитаем, сколько в таблице пропущенных значений. Для этого применим два метода `pandas`:

In [5]:
#выявление и подсчет пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски в `days_employed` и `total_income` являются важными, так как могут помешать объективной оценке кредитонадежности клиента. Если посмотреть на сумму этих пропусков, видно, что их количество совпадает. Таким образом, можно сделать вывод об отсутствии случайности в этих пропусках: те, кто не имеет трудового стажа, не раскрывает свой ежемесячный доход. Посмотрим на людей с отсутствующими значениями подробнее:

In [6]:
#сохранение в df_missed строк с пропущенными значениями
df_missed = df[df['days_employed'].isnull() & df['total_income'].isnull()]
df_missed.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Как видно, в данной таблице присутствуют в основном взрослые люди, при этом с указанным типом занятости. Скорее всего они предпочли либо скрыть свой стаж и доход, либо забыли указать это, либо их работа неофициальна, поэтому в таблице получились пропуски. Разберемся с ними по очереди. Для `days_employed` наиболее разумным будет заполнить пропуски нулями, чтобы наши значения не сдвигались в большую сторону и не изменяли качество данных.

Заполним пропуски в столбцах `days_employed` нулями. 

In [7]:
#заполнение пропущенных значений и сохранение операций в столбцах в переменной df
df['days_employed'] = df['days_employed'].fillna(0)

Теперь разберёмся с пропусками столбца `total_income`. Заполним эти пропуски на основе типа занятости человека. Для этого сначала найдём уникальные типы занятости:

In [8]:
#находим уникальные значения income_type
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Получили 8 типов занятости. Сгруппируем наши данные по этим типам и найдем медианные значения дохода по каждому:

In [9]:
#группируем данные и считаем медианы
total_income_medians = df.groupby('income_type')['total_income'].median()
#выводим медианные значения на экран
total_income_medians

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Заполним все пропущенные значения на основании полученных медианных значений:

In [10]:
#находим строки таблицы с пропуском в total_income и для каждого типа занятости заполняем уровень дохода медианой
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = total_income_medians[6]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = total_income_medians[4]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = total_income_medians[3]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = total_income_medians[2]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'безработный'), 'total_income'] = total_income_medians[0]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = total_income_medians[5]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'студент'), 'total_income'] = total_income_medians[7]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'в декрете'), 'total_income'] = total_income_medians[1]

Проверим результат. Для этого вновь посчитаем пропуски в таблице:

In [11]:
#проверка результатов повторным подсчетом пропусков
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Предобработка обнаружила пропуски в данных. Оставить их было бы возможным, если бы эти пропуски не находились в значимых для исследования переменных. В данном случае было необходимо заменить их нулями и медианныцми значениями. Без дубликов результаты исследования будут более точными. 

Далее можем перейти к замене типов данных.

### Замена типа данных

В ходе изучения данных было обнаружено, что столбцы `days_employed` и `total_income` имеют нехарактерные для них типы данных - float64. Эти данные были бы более удобными для изучения, если бы имели тип int64. Приведем их к нему.

Переведём значения столбцов `days_employed` и `total_income` из float64 в int64. Так как мы работаем с количественными значениями, применим метод `astype`:

In [12]:
#переводим значения в тип int64
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')                           

Посмотрим на типы данных в таблице после перевода:

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Оценим, на сколько проще стало осознавать эти переменные при просмотре таблицы данных:

In [14]:
#выводим пять первых строк таблицы df
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

В ходе просмотра данных обнаружились две переменные с несоответствующим им типами данных. На результаты исследования отсутствие таких исправлений никак бы не повлияло, однако следующему, кто будет работать с таблицей, будет гораздо проще в ней разбираться, не смотря на тысячные доли копеек или секунд, поэтому для анализа лучше приводить такие переменные в соответствующий вид.

Далее можно переходить к обработке дубликатов.

### Обработка дубликатов

Проверим, содержит ли наша таблица дубликаты. Для начала найдем явные дубликаты одной командой:

In [15]:
#подсчет явных дубликатов
df.duplicated().sum()

54

В таблице присутствует 54 явных дубликата. Возьпользуемся специальным методом `pandas` для их удаления, а после удаления изменим индексы полученных строк методом `reset_index`:

In [16]:
#удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

Еще раз посчитаем явные дубликаты, чтобы убедиться в их отсутствии:

In [17]:
#проверка на отсутствие дубликатов
df.duplicated().sum()

0

Взглянем на нашу таблицу еще раз, чтобы увидеть, возможно ли в ней наличие явных дубликатов:

In [18]:
#просмотр первых 10 строк таблицы df:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Как видно, в столбце `education` переменные хранятся в виде строковых значений, при этом некоторые из них имеют верхний регистр вместо нижнего. Если эта проблема присутствует в этом столбце, она может содержаться и в любом другом столбце со строковыми значениями. Для поиска дубликатов приведем все строковые столбцы к нижнему регистру:

In [19]:
#приведение всех столбцов со строковыми значениями к нижнему регистру
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

В столбце `gender` присутствуют строковые значения верхнего регистра. Во избежании недочета дубликатов приведем все значения этого столбца к верхнему регистру:

In [20]:
#приведение всех значений столбца к верхнему регистру
df['gender'] = df['gender'].str.upper()

Подсчитаем количество явных дубликатов в таблице после изменения регистров:

In [21]:
#подсчет явных дубликатов после изменений регистра
df.duplicated().sum()

17

Применим удаление и для этих строк-дубликатов:

In [22]:
#удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

**Вывод**

В ходе предобработке было найдено 71 пропусков в таблице, при этом присутствовали как явные, так и неявные дубликаты. Такие строки могли появиться из-за банальной человеческой ошибки, например, клиент подал заявку дважды. Для улучшения результатов исследования от таких строк пришлось избавиться при помощи специальных методов `pandas`, однако говорить об окончании предобработке пока рано - необходимо провести лемматизацию.

### Лемматизация

В столбце `purpose` клиенты описывают цель получения кредита. Несмотря на то, что цели часто бывают схожими, описываются они часто разными словами, из-за чего в столбце присутствуют разные, но фактически одинаковые значения. Чтобы избавиться от этого - необходимо воспользоваться лемматизацией. Импортируем из библиотеки для лемматизации функцию `Mystem`  для этой задачи:

In [23]:
#импортируем из библиотеки pymystem3 функцию Mystem
from pymystem3 import Mystem

К каждому элементу столбца `purpose` применим лемматизацию и посмотрим, на какие цели люди берут кредит. Для этого воспользуемся циклом `for`. Получившиеся значения сохраним в список `purpose_lemma_list`:

In [24]:
#создадим пустой лист purpose_lemma_list
purpose_lemma_list = []
#перебор элементов столбца purpose и их лемматизация
m = Mystem()
for elem in df['purpose']:
    lemma = m.lemmatize(elem)
    purpose_lemma_list.append(lemma)

Посмотрим на получившиеся леммы для оценки, на какие цели чаще всего берут кредит:

In [25]:
#список с леммами столбца purpose
purpose_lemma_list

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['недвижимость', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'п

**Вывод**

Стало ясно, что в целом люди берут кредиты на схожие цели, но говорят о них по-разному. В столбце `purpose` необходимо было провести лемматизацию, чтобы выявить эти категории. По результатам лемматизации становится ясно, что чаще всего люди берут кредит на следующие цели:
* Приобретение жилья / недвижимости
* Приобретение автомобиля
* Образование
* Свадьба

Перейдем к категоризации данных.

### Категоризация данных

Категоризуем наши данные по столбцу `purpose`, так как именно в этом столбце встречаются разные, но одинаковые по смыслу значения переменной. Для этого изменим название цели каждого человека на предложенные 4 категории, применяя поиск слов в названии цели. Для этого напишем функцию:

In [26]:
#создаем функцию, которая будет перебирать цели клиента и выводить для каждой переформулированную под стандарт цель
def purpose_group(purpose):
    #по ключевым словам найдем и заменим все цели
    if 'образовани' in purpose:
        return 'образование'
    if 'свадьб' in purpose:
        return 'свадьба'
    if 'жиль' in purpose or 'недвижим' in purpose:
        return 'приобретение жилья / недвижимости'
    if 'автомоби' in purpose:
        return 'приобретение автомобиля' 

Воспользуемся функцией `purpose_group` для создания столбца с новыми стандартизированными категориями:

In [27]:
#вызываем функцию и применяем ее к каждому значению столбца df['purpose']. Результат сохраняем в новый столбец
df['purpose_categ'] = df['purpose'].apply(purpose_group) 

Оценим состояние столбца `purpose_categ` на количество различных категорий:

In [28]:
#смотрим, сколько значений в какой категории оказались
df['purpose_categ'].value_counts()

приобретение жилья / недвижимости    10811
приобретение автомобиля               4306
образование                           4013
свадьба                               2324
Name: purpose_categ, dtype: int64

Остается еще один столбец, который было бы лучше категоризировать для последующей проверки гипотез - `total_income`. Для этого выделим 4 группы населения по доходам. Так как нам неизвестно заранее, на сколько в целом велики доходы по данным клиентам, обозначим эти группы условно следующим образом:
1. Низкий заработок
2. Средний заработок
3. Высокий заработок
4. Очень высокий заработок

Для такой разбивки воспользуем методом `quantile`:

In [29]:
df['total_income'].quantile([0.25,0.5,0.75])

0.25    107623.00
0.50    142594.00
0.75    195820.25
Name: total_income, dtype: float64

Создадим функцию, которая будет перебирать значения зарплаты клиента и на основании этого присвоит категорию:

In [30]:
#создаем функцию, которая будет перебирать зарплату клиентов и выводить категорию зарплаты клиента
def income_group(income):
    if 0 <= income <= 89088.50:
        return 'низкий заработок'
    if 89088.51 <= income <= 135781.00:
        return 'средний заработок'
    if 135781.01 <= income <= 195813.25:
        return 'высокий заработок'
    if 195813.26 <= income:
        return 'очень высокий заработок'

Воспользуемся функцией `income_group` для создания столбца с категорией зарплаты клиента:

In [31]:
#вызываем функцию и применяем ее к каждому значению столбца df['total_income']. Результат сохраняем в новый столбец
df['income_categ'] = df['total_income'].apply(income_group) 

Оценим состояние столбца `income_categ` на количество различных категорий:

In [32]:
#смотрим, сколько значений в какой категории оказались
df['income_categ'].value_counts()

высокий заработок          7079
средний заработок          5749
очень высокий заработок    5365
низкий заработок           3261
Name: income_categ, dtype: int64

Посмотрим на переменные образования и семейного статуса. Было бы лучше, если бы они хранились словарями. Обратимся к значениям этих столбцов:

In [33]:
#просматриваем количество уникальных значений образования
df['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [34]:
#просматриваем количество уникальных значений семейного статуса
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Посмотрим на два других столбца: `family_status_id` и `education_id` и на количество уникальных значений в них:

In [35]:
#просматриваем количество уникальных значений id семейного статуса
df['family_status_id'].value_counts()

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

In [36]:
#просматриваем количество уникальных значений id образования
df['education_id'].value_counts()

1    15172
0     5250
2      744
3      282
4        6
Name: education_id, dtype: int64

Как видно, сопоставив таблицы друг с другом, можно выделить 2 словаря:

In [37]:
family_dict = {'женат / замужем': 0, 'гражданский брак': 1, 'вдовец / вдова': 2, 'в разводе': 3, 'не женат / не замужем': 4}
education_dict = {'высшее': 0, 'среднее': 1, 'неоконченное высшее': 2, 'начальное': 3, 'ученая степень': 4}

В дальнейшем банк сможет использовать данные словари для своих целей. Как видно, выявить четкий алгоритм присвоения значений элементам словаря довольно трудно без статистических данных, поэтому пока будем считать, что значения присвоены случайным образом.

**Вывод**

Для проверки гипотез некоторые из представленных данных было необходимо категоризировать. К ним отнеслись столбцы `purpose`и `total_income`. Как видно, большая часть людей берет кредит на приобретение жилья и недвижимости, а на свадьбу берут реже всего. Доходы оказались довольно неожиданными. Более 90% клиентов зарабатывают существенно выше среднего показателя. 

Таким образом, этап предобработки можно считать завершенным. Можно переходить к вопросам исследования

## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Выясним, влияет ли наличие детей на возврат кредита в срок. Для этого сначала сгруппируем всех клиентов по количеству детей, а затем посчитаем для каждой группы задолженности. Для этого применим сводную таблицу:



In [38]:
#сделаем сводную таблицу по данным о количестве детей и наличии задолженности в прошлом
children_pivot = df.pivot_table(index=['children'], columns=['debt'], values='family_status_id', aggfunc='count')
#посмотрим на итоговую таблицу
children_pivot

debt,0,1
children,,
-1,46.0,1.0
0,13028.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN
20,68.0,8.0


В данной таблице расчитаем процент должников в каждой представленной группе. Для этого поделим количество должников на общее количество людей в каждой группе:

In [39]:
#находим процент должников в каждой группе и умножаем результат на 100 для получения процентов вместо долей
children_pivot['ratio'] = children_pivot[1] / children_pivot[0] * 100
#выводим результат на экран
children_pivot

debt,0,1,ratio
children,,,
-1,46.0,1.0,2.173913
0,13028.0,1063.0,8.159349
1,4364.0,444.0,10.174152
2,1858.0,194.0,10.441335
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN
20,68.0,8.0,11.764706


**Вывод**

Таким образом, видно, что те, у кого нет детей, чаще имеют задолженности по кредиту. Аналогично можно сказать и про тех, кто имеет 3 детей. В остальных случаях количество детей прямо пропорционально отсутствию кредитной задолженности.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выясним, влияет ли семейное положение `family_status` на возврат кредита в срок `debt`. Для этого  сначала сгруппируем все строки по семейному положению, а затем посчитаем для каждой группы задолженности. Для этого применим сводную таблицу:

In [40]:
#сделаем сводную таблицу по данным семейного положения и наличия задолженности в прошлом
family_status_pivot = df.pivot_table(index=['family_status'], columns=['debt'], values='family_status_id', aggfunc='count')
#посмотрим на итоговую таблицу
family_status_pivot

debt,0,1
family_status,,
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931
не женат / не замужем,2536,274


В данной таблице расчитаем процент должников в каждой представленной группе семейного положения. Для этого поделим количество должников на общее количество людей в каждой группе:

In [41]:
#находим процент должников в каждой группе и умножаем результат на 100 для получения процентов вместо долей
family_status_pivot['ratio'] = family_status_pivot[1] / family_status_pivot[0] * 100
family_status_pivot

debt,0,1,ratio
family_status,,,
в разводе,1110,85,7.657658
вдовец / вдова,896,63,7.031250
гражданский брак,3763,388,10.310922
женат / замужем,11408,931,8.160940
не женат / не замужем,2536,274,10.804416


**Вывод**

Получается очевидный вывод. Семейное положение влияет на задержку по оплате кредита: чаще имеют задержки люди в разводе либо вдовцы и вдовы. Это можно было бы объяснить тем, что одинокому человеку труднее исполнять обязанности по кредиту, но мы видим, что наилучшие показатели по отсутствию задержек имеют не женатые и не замужние. Исходя из этого, можно считать, что потерявшие своего партнера хуже исполняют обязательства по кредиту, чем те, кто его имеет, либо не имели вовсе. Предположим, что на возврат кредита в срок влияет уровень дохода, проанализировав его далее.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выясним, влияет ли уровень дохода `total_income` на возврат кредита в срок `debt`. Для этого  сначала сгруппируем все строки по категоризированному уровню дохода, а затем посчитаем для каждой группы задолженности. Для этого применим сводную таблицу:

In [42]:
#сделаем сводную таблицу по категоризированному доходу и наличию задолженности в прошлом
income_pivot = df.pivot_table(index=['income_categ'], columns=['debt'], values='total_income', aggfunc='count')
#посмотрим на итоговую таблицу
income_pivot

debt,0,1
income_categ,,
высокий заработок,6463,616
низкий заработок,3012,249
очень высокий заработок,4982,383
средний заработок,5256,493


В данной таблице расчитаем процент должников в каждой представленной группе. Для этого поделим количество должников на общее количество людей в каждой группе:

In [43]:
#находим процент должников в каждой группе и умножаем результат на 100 для получения процентов вместо долей
income_pivot['ratio'] = income_pivot[1] / income_pivot[0] * 100
#выводим результат на экран
income_pivot

debt,0,1,ratio
income_categ,,,
высокий заработок,6463,616,9.531177
низкий заработок,3012,249,8.266932
очень высокий заработок,4982,383,7.687676
средний заработок,5256,493,9.379756


**Вывод**

В данной ситуации имеем следующие выводы: чаще имеют задолженности по кредиту те, чей заработок является очень высоким, либо низким. Это можно объяснить довольно просто: те, чьи заработные платы не слишком велики, не всегда могут вовремя заплатить кредит, а те, чьи зарплаты слишком велики, либо берут кредиты на слишком большие суммы и тоже не могут вовремя их оплатить, либо просто забывают об этом. 

Чтобы оценить влияние целей кредита на возврат кредита в срок вернемся к созданному столбцу `df['purpose_categ']` и посмотрим распределения в нём:

In [44]:
df['purpose_categ'].value_counts()

приобретение жилья / недвижимости    10811
приобретение автомобиля               4306
образование                           4013
свадьба                               2324
Name: purpose_categ, dtype: int64

Выясним, влияет ли цель кредита на возврат кредита в срок. Для этого сначала сгруппируем все строки по категоризированной цели кредита, а затем посчитаем для каждой группы задолженности. Для этого применим сводную таблицу:

In [45]:
#сделаем сводную таблицу по категоризированным целям и наличию задолженности в прошлом
purpose_pivot = df.pivot_table(index=['purpose_categ'], columns=['debt'], values='total_income', aggfunc='count')
#посмотрим на итоговую таблицу
purpose_pivot

debt,0,1
purpose_categ,,
образование,3643,370
приобретение автомобиля,3903,403
приобретение жилья / недвижимости,10029,782
свадьба,2138,186


В данной таблице расчитаем процент должников в каждой представленной группе. Для этого поделим количество должников на общее количество людей в каждой группе:

In [46]:
#находим процент должников в каждой группе и умножаем результат на 100 для получения процентов вместо долей
purpose_pivot['ratio'] = purpose_pivot[1] / purpose_pivot[0] * 100
#выводим результат на экран
purpose_pivot

debt,0,1,ratio
purpose_categ,,,
образование,3643,370,10.156464
приобретение автомобиля,3903,403,10.325391
приобретение жилья / недвижимости,10029,782,7.797388
свадьба,2138,186,8.699719


**Вывод**

Видна некоторая зависимость. Те, кто берет кредит на образование или приобретение автомобиля, реже имеют задержку по оплате кредита, чем те, кто берет кредит на свадьбу или приобретение жилья, при этом кредит на недвижимость сопоставим с самой частой задержкой по оплате.

## Шаг 4. Общий вывод

Таким образом, исследование завершается. В ходе него мы выполнили 3 важнейших этапа исследования данных:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.
 
Проверка гипотез оказалась довольно продуктивной, поэтому системы кредитного скорринга можно будет улучшить при помощи полученных результатов. Как видно из исследования, имеем следующие выводы:
 1. Количество детей не влияет на факт погашения кредита в срок.
 2. Семейное положение влияет на факт погашения кредита в срок. Те люди, которые потеряли своего партнера, чаще других имеют задолженности по кредиту.
 3. Уровень заработной платы влияет на факт погашения кредита в срок. Чаще всего имеют задержки люди с низкой заработной платой, однако население, реже задерживающее оплату по кредиту, не прямо связано с ростом дохода. До определеннного среднего значения люди действительно имеют меньше задержек, однако затем с ростом дохода увеличивается вероятность задержки. Получается некоторое "перевернутое" нормальное распределение.
 4. Цель кредита влияет на факт погашения кредита в срок. Те, кто берет кредит на недвижимость или свадьбу, задержатся с большей вероятностью, чем те, кто берет кредит на образование или приобретение автомобиля.
 
 Таким образом, исследование можно полностью считать завершенным.
 